In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from brisc.manuscript_analysis import overview_image
from brisc.manuscript_analysis import cell_typing
from brisc.manuscript_analysis import load

from iss_preprocess.io import get_processed_path, load_ops, get_processed_path
from iss_preprocess.pipeline.sequencing import (
    basecall_tile,
    load_and_register_sequencing_tile,
)

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib
import iss_analysis as iss_analysis
import numpy as np
import scanpy as sc
from tqdm import tqdm


matplotlib.rcParams["pdf.fonttype"] = 42  # for pdfs

In [ ]:
# Load rabies and barcode rounds from tile
ops = load_ops("becalia_rabies_barseq/BRAC8498.3e/chamber_08")
rabies_rounds_stack, _, _ = basecall_tile(
    "becalia_rabies_barseq/BRAC8498.3e/chamber_08", (2, 2, 3), save_spots=False
)

genes_rounds_stack, _ = load_and_register_sequencing_tile(
    data_path="becalia_rabies_barseq/BRAC8498.3e/chamber_08",
    tile_coors=(2, 2, 3),
    filter_r=ops["filter_r"],
    prefix="genes_round",
    suffix=ops["genes_projection"],
    nrounds=ops["genes_rounds"],
    correct_channels=ops["genes_correct_channels"],
    corrected_shifts=ops["corrected_shifts"],
    correct_illumination=True,
)
genes_rounds_stack = genes_rounds_stack[:, :, np.argsort(ops["camera_order"]), :]

In [ ]:
image_dict = {}

vmax_dict = {
    "rab": (23000, 23000, 23000, 23000),
    "genes": (24000, 24000, 24000, 24000),
    "mCherry": (7500, 4000),
    "hyb": (5000, 3000, 7000, 2000),
    "hyb2": (100000, 100000, 100000, 8000),
}

vmin_dict = {
    "rab": (17000, 17000, 17000, 17000),
    "genes": (18600, 18600, 18600, 18600),
    "mCherry": (10, 10),
}

colors_dict = {
    "rab": ([0, 1, 1], [1, 0, 1], [0, 1, 0], [1, 0, 0]),
    "genes": ([0, 1, 1], [1, 0, 1], [0, 1, 0], [1, 0, 0]),
    "mCherry": ([1, 0, 0], [0, 1, 1]),
}

downsampling_dict = {
    "rab": 4,
    "genes": 2,
    "mCherry": 2,
}
for image in tqdm(["rab", "genes", "mCherry"]):
    processed_path = get_processed_path(
        f"becalia_rabies_barseq/BRAC8498.3e/chamber_08/manual_starter_click/BRAC8498.3e_chamber_08_2_{image}.tif"
    )
    print(f"Making downsampled RGB for {image} image")
    rgb = overview_image.make_downsampled_rgb(
        processed_path,
        downsample_factor=downsampling_dict[image],
        channel_colors=colors_dict[image],
        vmax=vmax_dict[image],
        vmin=vmin_dict[image],
    )
    image_dict[image] = rgb

In [ ]:
# Load cell clustering data for UMAPs
processed_path = get_processed_path(
    "becalia_rabies_barseq/BRAC8498.3e/analysis/adata_q.h5ad"
)
adata = sc.read_h5ad(processed_path)


processed_path = get_processed_path(
    "becalia_rabies_barseq/BRAC8498.3e/analysis/BRAC8498.3e_error_corrected_barcodes_26_cell_barcode_df.pkl"
)

cells_df = load.load_cell_barcode_data(
    processed_path,
    areas_to_empty=["fiber tracts", "outside"],
    valid_areas=["Isocortex", "TH"],
    distance_threshold=150,
)
filtered_ids = set(cells_df.index)
adata.obs["in_filtered_df"] = adata.obs.index.isin(filtered_ids)
adata.obs["in_filtered_df"].replace(False, np.nan, inplace=True)

In [ ]:
cm = 1 / 2.54
fontsize_dict = {"title": 7, "label": 8, "tick": 6, "legend": 6}
line_width = 0.5
line_alpha = 1

fig, axes = plt.subplot_mosaic(
    [
        ["mCherry", "mCherry", "rab", "rab", "rab", "rab"],
        ["genes", "genes", "rab", "rab", "rab", "rab"],
        [
            "rab_round1",
            "rab_round2",
            "UMAP_clust",
            "UMAP_clust",
            "UMAP_bar",
            "UMAP_bar",
        ],
        [
            "rab_round9",
            "rab_round10",
            "UMAP_clust",
            "UMAP_clust",
            "UMAP_bar",
            "UMAP_bar",
        ],
    ],
    figsize=(17.4 * cm, 13 * cm),
    dpi=300,
    gridspec_kw={
        "width_ratios": [1, 1, 1, 1, 1, 1],
        "height_ratios": [1, 1, 0.5, 0.5],
        "wspace": 0.1,
        "hspace": 0.1,
    },
)

inset_downsample_factor = 5
inset_frac = 0.40
offset_frac = 0.03
outer_col = "white"
inner_col = "white"
outer_lw = 1
inner_lw = 0.8

inner_specs = {
    "mCherry": {"tl": (0.28, 0.46), "h": 0.15},
    "genes": {"tl": (0.28, 0.46), "h": 0.15},
}
default_spec = {"tl": (0.30, 0.30), "h": 0.40}

scalebar_um = {"rab": 250, "mCherry": 50, "genes": 50}
x0, x1 = 0.307, 0.518
y0, y1 = 0.279, 0.485

# First do rab (to get aspect), then mCherry, then genes
rab_aspect = None
for key in ["rab", "mCherry", "genes"]:
    ax = axes[key]
    img = image_dict[key]
    H, W = img.shape[:2]

    # Coordinates of the outer crop (same for all three channels)
    x0_px = int(x0 * W)
    x1_px = int(x1 * W)
    y0_px = int(y0 * H)
    y1_px = int(y1 * H)

    # Rabies, no inset but draw dotted box on the main crop
    if key == "rab":
        main_crop = img[y0_px:y1_px, x0_px:x1_px]
        rab_aspect = (x1_px - x0_px) / (y1_px - y0_px)

        ax.imshow(main_crop, interpolation="none")

        # Draw dotted rectangle that marks the zoom-in used by mCherry/genes
        spec = inner_specs.get("mCherry", default_spec)
        tlx, tly = spec["tl"]
        h_frac = spec["h"]

        owH, owW = main_crop.shape[:2]
        box_h = int(h_frac * owH)
        box_w = int(box_h * rab_aspect)
        if box_w > owW:
            box_w = owW
            box_h = int(box_w / rab_aspect)

        x0_in = np.clip(int(tlx * owW), 0, owW - box_w)
        y0_in = np.clip(int(tly * owH), 0, owH - box_h)

        ax.add_patch(
            patches.Rectangle(
                (x0_in, y0_in),
                box_w,
                box_h,
                linewidth=inner_lw,
                edgecolor=inner_col,
                linestyle="dotted",
                facecolor="none",
            )
        )

        if key in scalebar_um:
            overview_image.add_scalebar(
                ax,
                downsample_factor=downsampling_dict[key],
                length_um=scalebar_um[key],
            )

        overview_image.print_image_stats(
            key,
            main_crop,
            pixel_size_um=0.231,
            downsample_factor=downsampling_dict[key],
        )
        ax.axis("off")
        continue

    outer_crop = img[y0_px:y1_px, x0_px:x1_px]

    # --------- dotted rectangle spec inside the OUTER crop --------
    spec = inner_specs.get(key, default_spec)
    tlx, tly = spec["tl"]
    h_frac = spec["h"]

    owH, owW = outer_crop.shape[:2]
    box_h = int(h_frac * owH)
    box_w = int(box_h * rab_aspect)
    if box_w > owW:  # keep aspect consistent
        box_w = owW
        box_h = int(box_w / rab_aspect)
    x0_in = np.clip(int(tlx * owW), 0, owW - box_w)
    y0_in = np.clip(int(tly * owH), 0, owH - box_h)

    # zoomed-in region from rabies for genes/mcherry panel
    main_crop = outer_crop[y0_in : y0_in + box_h, x0_in : x0_in + box_w]
    ax.imshow(main_crop, interpolation="none")

    # add scale-bar
    if key in scalebar_um:
        overview_image.add_scalebar(
            ax,
            downsample_factor=downsampling_dict[key],
            length_um=scalebar_um[key],
        )
    ax.axis("off")
    overview_image.print_image_stats(
        key,
        main_crop,
        pixel_size_um=0.231,
        downsample_factor=downsampling_dict[key],
    )

    # ------------------------------------------------------
    # INSET ─ show a **further 5× down-sampled** view of the
    #         SAME outer_crop; no interpolation this time.
    # ------------------------------------------------------
    outer_crop_ds = overview_image.downsample_xy(outer_crop, inset_downsample_factor)

    # inset axes placement identical to before
    bbox = ax.get_position()
    pw, ph = bbox.width, bbox.height
    iw, ih = pw * inset_frac, ph * inset_frac
    ix = bbox.x0 - pw * offset_frac
    iy = bbox.y1 + ph * offset_frac - ih
    axins = fig.add_axes([ix, iy, iw, ih])
    axins.set(xticks=[], yticks=[])

    # display *down-sampled* inset
    if key == "mCherry":
        axins.imshow(np.clip(outer_crop_ds / 0.5, 0, 1), interpolation="none")
    elif key == "genes":
        axins.imshow(np.clip(outer_crop_ds / 1.5, 0, 1), interpolation="none")

    # pretty border around inset
    for sp in axins.spines.values():
        sp.set_visible(True)
        sp.set_edgecolor(outer_col)
        sp.set_linewidth(outer_lw)

    # dotted rectangle marking zoom-in MUST be scaled to the
    # down-sampled resolution
    scale = 1 / inset_downsample_factor
    axins.add_patch(
        patches.Rectangle(
            (x0_in * scale, y0_in * scale),
            box_w * scale,
            box_h * scale,
            linewidth=inner_lw,
            edgecolor=inner_col,
            linestyle="dotted",
            facecolor="none",
        )
    )

cropped_rabies_cells = rabies_rounds_stack[1250:1550, 1750:2050, :, :]
overview_image.plot_selected_rounds(
    [
        axes["rab_round1"],
        axes["rab_round2"],
        axes["rab_round9"],
        axes["rab_round10"],
    ],
    cropped_rabies_cells,
    selected_rounds=[1, 2, 9, 10],
    fontsize=fontsize_dict["legend"],
    vmin=np.array([0, 0, 0, 0]),
    vmax=np.array([0.4, 0.4, 0.25, 0.3]),  # R, G, M, C
)

overview_image.add_scalebar(
    axes["rab_round10"],
    downsample_factor=1,
    length_um=20,
)

overview_image.print_image_stats(
    "rab_rounds",
    cropped_rabies_cells,
    pixel_size_um=0.231,
    downsample_factor=1,
)

cell_typing.plot_cell_clusters(
    adata,
    axes["UMAP_clust"],
    spot_size=1,
    fontsize=4,
    font_outline=0.5,
)
cell_typing.plot_umap_barcoded_cells(
    adata,
    axes["UMAP_bar"],
    size_non_barcoded=1,
    size_barcoded=0.5,
    legend_fontsize=6,
)

plt.savefig(
    "/nemo/lab/znamenskiyp/home/shared/presentations/becalick_2025/rabies_barseq_overview.pdf",
    dpi=600,
)